<a href="https://colab.research.google.com/github/haizzzi/simple-nmt-simplied/blob/main/NTC_Exe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch

# GPU가 있는지 확인
if torch.cuda.is_available():
    print("사용 가능한 GPU 수:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i} 이름:", torch.cuda.get_device_name(i))
else:
    print("GPU를 사용할 수 없습니다.")


사용 가능한 GPU 수: 1
GPU 0 이름: Tesla T4
